In [1]:
import tensorflow as tf
import numpy as np
import os

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 数据准备

In [2]:
from sklearn.preprocessing import StandardScaler


def unpickle(file):
    '''
    CIFAR-10数据读取函数
    '''
    import pickle
    with open(file, 'rb') as fd:
        data = pickle.load(fd, encoding='bytes')
    return data[b'data'], np.array(data[b'labels'])


class CifarData:
    def __init__(self, paths, batch_size=32, normalize=False, shuffle=False):
        '''
        paths: 文件路径
        '''
        self._data = list()
        self._target = list()
        self._n_samples = 0
        self.n_features = 0

        self._idx = 0    # mini-batch的游标
        self._batch_size = batch_size

        self._load(paths)

        if shuffle:
            self._shuffle_data()
        if normalize:
            self._normalize_data()

        print(self._data.shape, self._target.shape)

    def _load(self, paths):
        '''
        载入数据
        '''
        for path in paths:
            data, labels = unpickle(path)
            self._data.append(data)
            self._target.append(labels)

        # 将所有批次的数据拼接起来
        self._data, self._target = np.vstack(
            self._data), np.hstack(self._target)

        self._n_samples, self.n_features = self._data.shape[0], self._data.shape[1]

    def _shuffle_data(self):
        '''
        打乱数据
        '''
        idxs = np.random.permutation(self._n_samples)
        self._data = self._data[idxs]
        self._target = self._target[idxs]

    def _normalize_data(self):
        scaler = StandardScaler()
        self._data = scaler.fit_transform(self._data)

    def next_batch(self):
        '''
        生成mini-batch
        '''
        while self._idx < self._n_samples:
            yield self._data[self._idx: (self._idx+self._batch_size)], self._target[self._idx: (self._idx+self._batch_size)]
            self._idx += self._batch_size

        self._idx = 0
        self._shuffle_data()

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
CIFAR_DIR = "../dataset/cifar-10-batches-py/"
train_filenames = [os.path.join(
    CIFAR_DIR, 'data_batch_{}'.format(i)) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

batch_size = 32
train_data = CifarData(train_filenames, batch_size=batch_size,
                       normalize=True, shuffle=True)
test_data = CifarData(test_filenames, batch_size=batch_size,
                      normalize=True, shuffle=False)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(50000, 3072) (50000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(10000, 3072) (10000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# 网络结构设计
mobilenet中DSC+PC的结构是隔层使用的，即conv+DSC+PC+conv这样的结构。这里实现的mini-mobilenet与Googlenet类似，只使用两次级联结构。

In [4]:
unit_I = train_data.n_features    # 输入单元数，等于特征数

# 三种卷积核
conv_size1 = (1, 1)
conv_size3 = (3, 3)

filters_1 = 32
filters_2 = 64
filters_3 = 128

# 两种步长
strides_1 = (1, 1)
strides_2 = (2, 2)

fc_size = 128    # 全连接层单元数

unit_O = 10

定义创建```Conv dw```结构的函数：

In [5]:
def conv_dw(inputs, filters, strides, name=None):
    '''
    filters: 改变通道数，PC层使用的参数
    strides: 改变尺寸，DSC层使用的参数
    '''
    in_channels = inputs.get_shape().as_list()[-1]    # 最后一维为深度
    input_split = tf.split(inputs, in_channels, axis=3)    # 按深度进行分割
    DSC_outputs = list()

    with tf.name_scope(name):
        for i in range(in_channels):    # 分别对每个深度做卷积
            cur_conv = tf.layers.conv2d(input_split[i], filters=1,
                                        kernel_size=conv_size3, strides=strides,
                                        padding='same', activation=tf.nn.relu)
            DSC_outputs.append(cur_conv)
        DSC_outputs = tf.concat(DSC_outputs, axis=3)

        PC_layer = tf.layers.conv2d(DSC_outputs, filters=filters,
                                    kernel_size=conv_size3, strides=strides_1,
                                    padding='same', activation=tf.nn.relu)
        
    return PC_layer

# 搭建网络

In [6]:
X = tf.placeholder(tf.float32, [None, unit_I])  # 数据的样本数不指定，只指定特征数
Y = tf.placeholder(tf.int64, [None])    # 目标值为列向量，int64为了兼容
X_img = tf.transpose(tf.reshape(X, [-1, 3, 32, 32]),
                     perm=[0, 2, 3, 1])    # 转为图片格式送入模型，(n_samples,width,height,depth)

# 网络结构图
with tf.name_scope('MobileNet'):
    conv1 = tf.layers.conv2d(X_img, filters=filters_1,
                             kernel_size=conv_size3, strides=strides_2,
                             padding='same', activation=tf.nn.relu, name='conv1')
    conv_dw_1 = conv_dw(conv1, filters=filters_1,
                        strides=strides_1, name='conv_dw1')

    conv2 = tf.layers.conv2d(X_img, filters=filters_2,
                             kernel_size=conv_size1, strides=strides_1,
                             padding='same', activation=tf.nn.relu, name='conv2')
    conv_dw_2 = conv_dw(conv1, filters=filters_2,
                        strides=strides_2, name='conv_dw2')

    conv3 = tf.layers.conv2d(X_img, filters=filters_3,
                             kernel_size=conv_size3, strides=strides_1,
                             padding='same', activation=tf.nn.relu, name='conv3')
    conv_dw_3 = conv_dw(conv1, filters=filters_3,
                        strides=strides_1, name='conv_dw3')
    
    pooling=tf.layers.average_pooling2d(conv_dw_3, pool_size=conv_size3,
                                           strides=strides_1, name='pooling')

    fc = tf.layers.dense(tf.layers.flatten(pooling), fc_size,
                         activation=tf.nn.relu, name='FC')

    logits = tf.layers.dense(fc, unit_O, activation=None)

# 评估图
with tf.name_scope('Eval'):
    # 计算一维向量与onehot向量之间的损失
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
    predict = tf.argmax(logits, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), tf.float32))

# 优化图
with tf.name_scope('train_op'):
    lr = 1e-3
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True    # 按需使用显存

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# 训练网络

In [7]:
with tf.Session(config=config) as sess:
    sess.run(init)
    epochs = 20

    batch_cnt = 0
    for epoch in range(epochs):
        for batch_data, batch_labels in train_data.next_batch():
            batch_cnt += 1
            loss_val, acc_val, _ = sess.run(
                [loss, accuracy, train_op],
                feed_dict={
                    X: batch_data,
                    Y: batch_labels})

            # 每1000batch输出一次信息
            if (batch_cnt+1) % 1000 == 0:
                print('epoch: {}, batch_loss: {}, batch_acc: {}'.format(
                    epoch, loss_val, acc_val))

            # 每5000batch做一次验证
            if (batch_cnt+1) % 5000 == 0:
                all_test_acc_val = list()
                for test_batch_data, test_batch_labels in test_data.next_batch():
                    test_acc_val = sess.run(
                        [accuracy],
                        feed_dict={
                            X: test_batch_data,
                            Y: test_batch_labels
                        })
                    all_test_acc_val.append(test_acc_val)
                test_acc = np.mean(all_test_acc_val)
                print('epoch: {}, test_acc: {}'.format(epoch, test_acc))

epoch: 0, batch_loss: 1.1020231246948242, batch_acc: 0.625
epoch: 1, batch_loss: 0.9442151784896851, batch_acc: 0.6875
epoch: 1, batch_loss: 0.48784172534942627, batch_acc: 0.875
epoch: 2, batch_loss: 0.5022556781768799, batch_acc: 0.78125
epoch: 3, batch_loss: 0.7154605984687805, batch_acc: 0.78125
epoch: 3, test_acc: 0.7221445441246033
epoch: 3, batch_loss: 0.5692753195762634, batch_acc: 0.78125
epoch: 4, batch_loss: 0.3449956178665161, batch_acc: 0.84375
epoch: 5, batch_loss: 0.30386796593666077, batch_acc: 0.875
epoch: 5, batch_loss: 0.4960312843322754, batch_acc: 0.71875
epoch: 6, batch_loss: 0.2550877332687378, batch_acc: 0.90625
epoch: 6, test_acc: 0.7420127987861633
epoch: 7, batch_loss: 0.08763671666383743, batch_acc: 0.9375
epoch: 7, batch_loss: 0.05528689920902252, batch_acc: 0.96875
epoch: 8, batch_loss: 0.07687079906463623, batch_acc: 0.96875
epoch: 8, batch_loss: 0.2145913690328598, batch_acc: 0.9375
epoch: 9, batch_loss: 0.1558322012424469, batch_acc: 0.9375
epoch: 9, te